<a href="https://colab.research.google.com/github/summerolmstead/EconomicsLLM/blob/main/PreprocessingEconTxtBookPdfToTxt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.6 MB/s eta 0:00:00


In [3]:
import pdfplumber
import os

pdf_path = "/content/drive/MyDrive/EconLLM/Macroeconomics2.pdf"

with pdfplumber.open(pdf_path) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()  #extract text from each page

folder_path = os.path.dirname(pdf_path)

output_text_path = os.path.join(folder_path, "extracted_text.txt")

#save the extracted text to the output text file
with open(output_text_path, "w") as text_file:
    text_file.write(text)

#print success message with the file path
print(f"Text file saved to: {output_text_path}")

#show a snippet of the extracted text to see what it looks like
print(text[:1000])


Text file saved to: /content/drive/MyDrive/EconLLM/extracted_text.txt
Principles of
Macroeconomics 2e
SENIOR CONTRIBUTING AUTHORS
STEVEN A. GREENLAW, UNIVERSITY OF MARY WASHINGTON
DAVID SHAPIRO, PENNSYLVANIA STATE UNIVERSITY
Based on the 2nd edition of Principles of
Economics, Economics and the Economy, 2e
by Timothy Taylor, published in 2011.OpenStax
Rice University
6100 Main Street MS-375
Houston, Texas 77005
To learn more about OpenStax, visit https://openstax.org.
Individual print copies and bulk orders can be purchased through our website.
©2018 Rice University. Textbook content produced by OpenStax is licensed under a Creative Commons
Attribution 4.0 International License (CC BY 4.0). Under this license, any user of this textbook or the textbook
contents herein must provide proper attribution as follows:
- If you redistribute this textbook in a digital format (including but not limited to PDF and HTML), then you
must retain on every page the following attribution:
“Download for f

In [4]:
#ok so now i am going to move the extracted text i deleted the intro and need ome reg ex to clean the data and take out some parts.... ome sentenances are merged into a word so without
#spaces as well so i will try to fix all of those as well!


In [5]:
import re

#function to clean up merged words, numbers, and other formatting issues
def remove_unwanted_content2(text):

    #remove table references (e.g., "Table 4.6")
    text = re.sub(r'Table \d+\.\d+', '', text)

    #remove this specific sentence that shows up often
    text = re.sub(r'This OpenStax book is available for free at http[s]?://\S+', '', text)

    #remove URLs (e.g., "http://cnx.org/content/col12190/1.4") greedy regex
    text = re.sub(r'http[s]?://\S+', '', text)

    #clean up any excessive whitespace
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space

    #trim leading and trailing spaces
    text = text.strip()

    return text

In [6]:
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=bb46cc3d0d3abbddd6dd51eea7cc7816b08af39c3c9169db359aae4bbeac89de
  Stored in directory: /root/.cache/pip/wheels/aa/44/3a/f2a5c1859b8b541ded969b4cd12d0a58897f12408f4f51e084
Successfully built wordninja


In [7]:
import wordninja

#function to fix merged words using wordninja
def fix_merged_words_with_ninja(text):
    cleaned_text = ' '.join(wordninja.split(text))  #split the words and join them with a space

    return cleaned_text

#cxample of merged word text
merged_text = "Economicsisconcernedwiththewell-beingofallpeoplewhohavenotjobs"

#apply the fix_merged_words_with_ninja function to the merged text
cleaned_text = fix_merged_words_with_ninja(merged_text)

#print the cleaned text
print(cleaned_text)



Economics is concerned with the well being of all people who have not jobs


In [8]:
#path for editing output of manually deleted intro and end
txtfileedit1_path = "/content/drive/MyDrive/EconLLM/MacroeconomicsTxtBookEdit1.txt"

#step 1: Read the text from the file !!!!
with open(txtfileedit1_path, "r") as file:
    text = file.read()

#step 2: Apply the cleaning functions we just defined keeping chapter and subsection for easier training process
cleaned_text = remove_unwanted_content2(text)
cleaned_text = fix_merged_words_with_ninja(cleaned_text)

#step 3: Save the cleaned text back to a file in the same folder
output_path = "/content/drive/MyDrive/EconLLM/MacroeconomicsTxtBookEdit4.txt"

with open(output_path, "w") as file:
    file.write(cleaned_text)

#success message
print(f"Cleaned text saved to: {output_path}")

Cleaned text saved to: /content/drive/MyDrive/EconLLM/MacroeconomicsTxtBookEdit4.txt


Decided To train sections probably should keep chapters and subsections for the easiest way to automate all data for model 12/27

In [18]:
import re
import os

def parse_text_to_chapters_and_subsections(text):
    # regex pattern to match chapter headings (e.g., "chapter 1 introduction")
    chapter_pattern = r"(Chapter \d+ [^\n]+)"

    # regex pattern to match subsections like "1 5", "2 1", etc.
    subsection_pattern = r"(\d+ \d+ [^\n]+)"

    # split the text into potential chapters based on the chapter heading pattern
    split_chapters = re.split(f"({chapter_pattern})", text)

    chapter_data = {}
    current_chapter = None
    current_subsection = None

    print(f"total parts split: {len(split_chapters)}")  # debugging: check how many parts were split

    for part in split_chapters:
        part = part.strip()  # clean up the part

        # debugging: check each part being processed
        print(f"processing: {part[:30]}...")  # show the first 30 characters for context

        # check if this part matches a chapter heading
        if re.match(chapter_pattern, part):
            current_chapter = part
            chapter_data[current_chapter] = []  # initialize a list for the chapter content
            current_subsection = None  # reset subsection when a new chapter starts
        # check if this part matches a subsection heading
        elif re.match(subsection_pattern, part):
            current_subsection = part
            # append the subsection under the current chapter as a dictionary
            chapter_data[current_chapter].append({current_subsection: []})
        else:
            # if the part is content, append it under the current subsection (if active)
            if current_subsection:
                # append content to the current subsection
                chapter_data[current_chapter][-1][current_subsection].append(part)
            elif current_chapter:
                # if no subsection is active, append content directly to the chapter
                chapter_data[current_chapter].append(part)

    print(f"parsed chapters: {len(chapter_data)}")  # debugging: check how many chapters were parsed
    return chapter_data

# read the text from the input file
def read_input_file(input_path):
    if not os.path.exists(input_path):
        print(f"error: file {input_path} not found!")
        return ""

    with open(input_path, "r") as file:
        text = file.read()

    print(f"read {len(text)} characters from input file.")  # debugging: check how much text was read
    return text

# function to write the parsed data to a text file
def write_parsed_data_to_text(parsed_data, output_path):
    with open(output_path, "w") as file:
        if not parsed_data:
            print("warning: no data to write.")  # debugging: check if data is empty
        for chapter, subsections in parsed_data.items():
            file.write(f"{chapter}\n")
            for subsection_data in subsections:
                # handle subsections
                if isinstance(subsection_data, dict):
                    for subsection, content in subsection_data.items():
                        file.write(f"  {subsection}\n")
                        for item in content:
                            file.write(f"    {item}\n")
                else:
                    # if it's not a dictionary (i.e., it's content directly under the chapter)
                    file.write(f"  {subsection_data}\n")
            file.write("\n")  # newline to separate chapters

# path to input and output files
input_path = "/content/drive/MyDrive/EconLLM/MacroeconomicsTxtBookEdit4.txt"
output_path = "/content/drive/MyDrive/EconLLM/MacroeconomicsProcessSectionsResult3.txt"

# read the input text
text = read_input_file(input_path)

# if the text is valid, parse it into chapters and subsections
if text:
    parsed_data = parse_text_to_chapters_and_subsections(text)
    # write the parsed data to the output file
    write_parsed_data_to_text(parsed_data, output_path)

    # success message
    print(f"parsed data saved to: {output_path}")
else:
    print("no content read from the file. aborting further processing.")



read 1644234 characters from input file.
total parts split: 7
processing: Senior contributing authors St...
processing: Chapter 1 Welcome to Economics...
processing: Chapter 1 Welcome to Economics...
processing: ...
processing: Chapter 1 Welcome to Economics...
processing: Chapter 1 Welcome to Economics...
processing: ...
parsed chapters: 2
parsed data saved to: /content/drive/MyDrive/EconLLM/MacroeconomicsProcessSectionsResult3.txt
